In [126]:
import sys
sys.path.append("../src")
import pandas as pd

from preprocess import Preprocessing

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

# Import datas

In [3]:
train_df = pd.read_csv("../datas/train_data.csv")

# Only keep the sex column

In [128]:
train = train_df.drop(["date_accident", "date_consolidation", "ID", "filename"], axis=1)

train = train[train["sexe"] != "n.c."]


# Clean datas

In [129]:
preprocess_train = Preprocessing(train)

train = preprocess_train.remove_newlines()
train = preprocess_train.remove_stopwords()

train.head()


,texte,sexe
0,Le : 12/11/2019 Cour d’appel d’Agen chambre so...,homme
1,Le : 12/11/2019 Cour d’appel d’Agen chambre ci...,homme
2,Le : 12/11/2019 Cour d’appel d’Agen Audience p...,femme
3,Le : 12/11/2019 Cour d’appel d’Agen Audience p...,femme
4,Le : 12/11/2019 Cour d’appel d’Agen Audience p...,homme


# Let's predict the sex of the victim!

## We will use a TF-IDF

In [132]:
vect = TfidfVectorizer(
  max_features=5000,
  min_df=350,
  stop_words=list(fr_stop),
  ngram_range=(1, 2),
  binary=True)

train["sexe"] = train["sexe"].replace({'homme': 0,
                                       "femme": 1})

X_train, X_test, y_train, y_test = train_test_split(train["texte"], train['sexe'], test_size=0.2, random_state=42, stratify=train['sexe'])

X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

clf = LogisticRegression(C=1.5, random_state=1).fit(X_train, y_train)

preds = clf.predict((X_test))

/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_49079/3140563592.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train["sexe"] = train["sexe"].replace({'homme': 0,
/Users/SamuelLP/Desktop/git/projet_nlp/.nlp_venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(
/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_49079/3140563592.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future be

# Now, we compute our metrics

## On the test dataset

In [134]:
print("accuracy du test:", round(accuracy_score(y_test, preds), 2) * 100, "%")
print("F1 du test:", round(f1_score(y_test, preds, average='macro'), 2) * 100, "%")

accuracy du test: 74.0 %
F1 du test: 45.0 %
